<a href="https://colab.research.google.com/github/SergioManuelJob/EvolutiveAlgorithm-Exercise/blob/main/AlgoritmosEvolutivos_Sergio_Manuel_Su%C3%A1rez_Su%C3%A1rez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import random

ciudades = pd.read_csv('/content/drive/MyDrive/Ciudades.csv')
ciudades.head()

,Unnamed: 0,Las Palmas,Telde,Gáldar,Tamaraceite,Santa Brígida
0,Las Palmas,0,54,31,78,12
1,Telde,54,0,87,45,66
2,Gáldar,31,87,0,23,91
3,Tamaraceite,78,45,23,0,77
4,Santa Brígida,12,66,91,77,0


## Del dataframe de las distancias de las ciudades, nos quedamos solo con la información de las distancias entre las ciudades. Para eso está este bloque de código. Y lo mete en el array distancias_ciudades, que es una matriz que utilizamos para calcular la distancia total en la función calcularDistancia().

In [3]:
distancias_ciudades = ciudades.iloc[:, 1:].values
distancias_ciudades

array([[ 0, 54, 31, 78, 12],
       [54,  0, 87, 45, 66],
       [31, 87,  0, 23, 91],
       [78, 45, 23,  0, 77],
       [12, 66, 91, 77,  0]])

## Función para calcular la distancia total de los caminos, va recorriendo de ciudad en ciudad sumando las distancias de todos los caminos y devolviendo el total.



In [4]:
def calcularDistancia(camino):
    distancia = 0
    for i in range(0, 4, 1):
        ciudad_actual = camino[i]
        ciudad_siguiente = camino[i+1]
        distancia += distancias_ciudades[ciudad_actual, ciudad_siguiente]
        # print("Se ha sumado " + str(distancias_ciudades[indice_actual, indice_siguiente]) + " en la vuelta numero " + str(i+1))
    return distancia

## Ya que estamos ante un problema de minimización, al evaluar el fitness lo que queremos es conseguir el camino que menos distancia recorra para ir a todas las ciudades.
## Por tanto, el mejor individuo será el que tenga el valor más pequeño de distancia recorrida, y ese será el que se devuelve

In [5]:
def evaluarFitness(array):
  arrayCopia = np.copy(array)
  arrayCopia = list(arrayCopia) # Lo transformamos a lista para conseguir el metodo pop()
  mejorIndividuo = arrayCopia.pop()
  i = 0
  while i < len(arrayCopia):
    if(calcularDistancia(mejorIndividuo) > calcularDistancia(arrayCopia[i])):
      mejorIndividuo = arrayCopia[i]
    i += 1
  return mejorIndividuo

## Método para crear hijos a través de una recombinación donde el segundo padre señala que valores del padre número 1 hay que ir poniendo en el hijo, como visto en el vídeo.

In [6]:
def cruce(padre1, padre2):
  hijo = []
  for i in range(0, 5, 1):
    index = padre2[i]
    hijo.append(padre1[index])
  return hijo

## Este metodo intercambia los valores de dos indices aleatorios de un vector, con una probabilidad fija que se le pasa. Este metodo se utiliza para cambiar a los dos mejores individuos de forma leve y mantener la consistencia de los caminos, para seguir creando generaciones con ellos.

In [19]:
def intercambiar_indices(vector, probabilidad):
    vectorNuevo = np.copy(vector)
    if np.random.rand() < probabilidad:
        # Elegir dos índices aleatorios para intercambiar
        idx1, idx2 = np.random.choice(len(vector), 2, replace=False)

        # Intercambiar los valores en los índices seleccionados
        vectorNuevo[idx1], vectorNuevo[idx2] = vectorNuevo[idx2], vectorNuevo[idx1]

    return vectorNuevo

## Y esta es la función que hace todo el algoritmo evolutivo, coge una población inicial de 2 (P = 2) que muta más tarde con los dos mejores individuos con una probabilidad del 50% que se puede cambiar (M = 0.5), se crean dos hijos de esta población con una probabilidad de recombinación del 100%  también (R = 1.0), y de todo este conjunto de padres e hijos recoge a los 2 mejores en el metodo de Fitness (F = 2) y los vuelve a iterar en la función la cantidad de generaciones que el usuario decida, G es el parámetro que se le pasa a la función, G = n por tanto.

In [27]:
def buscarCamino(generaciones, individuos = None):

  if generaciones == 0:
    return individuos if individuos else []

  if individuos == None:
    individuos = []
    # Creamos dos padres, que son la población inicial
    for i in range (len(individuos)-2, len(individuos), 1):
      camino = np.array([0, 1, 2, 3, 4])
      random.shuffle(camino)
      individuos.append(camino)

  for i in range (len(individuos)-2, len(individuos), 1):
    camino = individuos[i]
    caminoNuevo = intercambiar_indices(camino, 0.5)
    individuos.append(caminoNuevo)



  # Aquí vemos los padres que son la población inicial y la distancia que tienen
  # for i in range(0, len(individuos), 1):
  #   print("La mutación número " + str(i+1) + " es: " + str(individuos[i]))
  #   print("Y su distancia es: " + str(calcularDistancia(individuos[i])))

  # Y de los dos el que es mejor se calcula con la función de fitness y se devuelve, tal que así:
  # print("\nY el mejor de los individuos es la combinación: " + str(evaluarFitness(individuos)))


  # Seleccionamos a los dos padres y con ellos los cruzamos y creamos dos hijos, que son así la nueva generación
  padre1 = individuos[0]
  padre2 = individuos[1]
  nuevaGeneracion = []
  nuevaGeneracion.append(cruce(padre1, padre2))
  nuevaGeneracion.append(cruce(padre2, padre1))
  print("\nVEMOS LOS RESULTADOS DE LA GENERACIÓN NÚMERO: " + str(generaciones)+ "\n")

  # Y aquí evaluamos a los de la nueva generación que salieron por recombinación
  # for i in range(0, len(nuevaGeneracion), 1):
  #   print("El hijo número " + str(i+1) + " es: " + str(nuevaGeneracion[i]))
  #   print("Y su distancia es: " + str(calcularDistancia(nuevaGeneracion[i])))

  # print("\nY el mejor de los dos es: " + str(evaluarFitness(nuevaGeneracion)))

  # Ahora vamos a escoger a los dos mejores de ahora, para eso primero los combinamos en un array

  arrayCombinado = []
  for i in range(len(individuos)):
    arrayCombinado.append(individuos[i])

  for i in range(len(nuevaGeneracion)):
    arrayCombinado.append(nuevaGeneracion[i])

  # Añadimos el mejor de esta generación al array, lo borramos de la lista y añadimos también al segundo mejor
  mejoresIndividuos = []
  mejoresIndividuos.append(evaluarFitness(arrayCombinado))
  arrayBorrar = evaluarFitness(arrayCombinado)
  indices = np.where([np.array_equal(arr, arrayBorrar) for arr in arrayCombinado])[0]
  arrayCombinado.pop(indices[0])
  mejoresIndividuos.append(evaluarFitness(arrayCombinado))

  print("\nLos mejores de momento son: " + str(mejoresIndividuos))
  print("Y el mejor de todos es " + str(evaluarFitness(mejoresIndividuos)) + " con " + str(calcularDistancia(evaluarFitness(mejoresIndividuos)))+ " de COSTE\n")

  buscarCamino(generaciones - 1, mejoresIndividuos)


## En esta línea llama a la función del algoritmo evolutivo, y pasa por parámetro la cantidad de generaciones que quieras hacer. Esta función es recursiva, y va llamando a la función para cada generación. Al final, devuelve el mejor camino que ha encontrado.

In [32]:
# El parámetro corresponde con la cantidad de generaciones que quieras hacer
buscarCamino(10)


VEMOS LOS RESULTADOS DE LA GENERACIÓN NÚMERO: 10


Los mejores de momento son: [array([4, 1, 0, 3, 2]), array([4, 1, 0, 3, 2])]
Y el mejor de todos es [4 1 0 3 2] con 221 de COSTE


VEMOS LOS RESULTADOS DE LA GENERACIÓN NÚMERO: 9


Los mejores de momento son: [array([4, 1, 0, 3, 2]), array([4, 1, 0, 3, 2])]
Y el mejor de todos es [4 1 0 3 2] con 221 de COSTE


VEMOS LOS RESULTADOS DE LA GENERACIÓN NÚMERO: 8


Los mejores de momento son: [array([4, 1, 3, 0, 2]), array([4, 1, 0, 3, 2])]
Y el mejor de todos es [4 1 3 0 2] con 220 de COSTE


VEMOS LOS RESULTADOS DE LA GENERACIÓN NÚMERO: 7


Los mejores de momento son: [array([4, 1, 3, 0, 2]), array([4, 1, 3, 0, 2])]
Y el mejor de todos es [4 1 3 0 2] con 220 de COSTE


VEMOS LOS RESULTADOS DE LA GENERACIÓN NÚMERO: 6


Los mejores de momento son: [array([4, 1, 3, 0, 2]), array([4, 1, 3, 0, 2])]
Y el mejor de todos es [4 1 3 0 2] con 220 de COSTE


VEMOS LOS RESULTADOS DE LA GENERACIÓN NÚMERO: 5


Los mejores de momento son: [array([3, 1, 4